NLP 공부
# BOW

In [1]:
import numpy as np

# 예시 문장들
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

# 전체 단어장 생성
words = set()
for doc in corpus:
    words.update(doc.lower().split())
words = sorted(words) # 리스트로변환

# 각 문서의 BoW 생성
bow = np.zeros((len(corpus), len(words)), dtype=int)
for i, doc in enumerate(corpus):
    for word in doc.lower().split():
        bow[i, words.index(word)] += 1 # list.index는 위치찾아주는걸 깜빡해서..

# 단어 인덱스 출력
print('단어 인덱스:')
word_index = {word: idx for idx, word in enumerate(words)}
print(word_index)
print()

# 문서별 BoW 출력
print('문서별 BoW:')
print(bow)


단어 인덱스:
{'and': 0, 'document': 1, 'document.': 2, 'document?': 3, 'first': 4, 'is': 5, 'one.': 6, 'second': 7, 'the': 8, 'third': 9, 'this': 10}

문서별 BoW:
[[0 0 1 0 1 1 0 0 1 0 1]
 [0 1 1 0 0 1 0 1 1 0 1]
 [1 0 0 0 0 1 1 0 1 1 1]
 [0 0 0 1 1 1 0 0 1 0 1]]


# Document-Term Matrix (DTM)


In [2]:
import numpy as np
import pandas as pd
import re

# 문서 데이터
documents = [
    "John likes to watch movies. Mary likes movies too.",
    "John also likes to watch football games.",
    "Mary plays football."
]

# 정규표현식을 사용하여 단어 추출
words = set()
for doc in documents:
    words.update(re.findall(r'\b\w+\b', doc.lower()))  # 정규표현식을 사용하여 단어 추출 및 소문자 변환
    # /b는 단어경계를 나타내는 문자 공백 문장부호 , /w 하나이상의 단어 문자 알파벳 숫자 포함
    # re.findall은 정규표현식에 맞는 모든 부분을 리스트로 변환하는 것
word_to_index = {word: idx for idx, word in enumerate(sorted(words))}

# DTM 초기화 (문서 수 x 단어 수)
dtm = np.zeros((len(documents), len(words)), dtype=int)

# 각 문서에서 단어 등장 횟수 계산
for doc_idx, doc in enumerate(documents):
    word_counts = {}
    for word in re.findall(r'\b\w+\b', doc.lower()):
        if word in word_to_index:
            word_idx = word_to_index[word] #여기서 word_idx에 여 왜 word_to_index워드 값을 넣는지 이해가 안 됐다.
            if word_idx not in word_counts: # 그냥 word_count에 있는지 확인하기 위함이었꼬 여기서 코드를 단순화 가능할 것 같기는한데
                word_counts[word_idx] = 0
            word_counts[word_idx] += 1

    for word_idx, count in word_counts.items(): # for 문이 여러개일때 집중력이 떨어지는게 약점
        dtm[doc_idx, word_idx] = count

# 각 단어별 전체 문서에서의 등장 횟수를 나타내는 '총합' 열 추가
total_counts = np.sum(dtm, axis=0)
dtm_with_total = np.vstack([dtm, total_counts])

# 단어 목록을 기반으로 pandas DataFrame 생성
columns = sorted(words)
index = ['문서1', '문서2', '문서3', '총합']
dtm_df = pd.DataFrame(dtm_with_total, columns=columns, index=index)

print(dtm_df)


     also  football  games  john  likes  mary  movies  plays  to  too  watch
문서1     0         0      0     1      2     1       2      0   1    1      1
문서2     1         1      1     1      1     0       0      0   1    0      1
문서3     0         1      0     0      0     1       0      1   0    0      0
총합      1         2      1     2      3     2       2      1   2    1      2


# TF- IDF (numpy 포기해야지)

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# 문서 데이터
documents = [
    "John likes to watch movies. Mary likes movies too.",
    "John also likes to watch football games.",
    "Mary plays football."
]

# CountVectorizer를 사용하여 DTM 생성
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

# TF-IDF 변환기 생성 및 TF-IDF 계산
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X)

# TF-IDF를 pandas DataFrame으로 변환
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out(), index=['문서1', '문서2', '문서3'])

# 각 단어별 전체 문서에서의 TF-IDF 값을 나타내는 '총합' 열 추가
tfidf_df.loc['총합'] = tfidf_df.sum(axis=0)

print(tfidf_df)


         also  football     games      john     likes      mary    movies  \
문서1  0.000000  0.000000  0.000000  0.245112  0.490223  0.245112  0.644585   
문서2  0.452123  0.343851  0.452123  0.343851  0.343851  0.000000  0.000000   
문서3  0.000000  0.517856  0.000000  0.000000  0.000000  0.517856  0.000000   
총합   0.452123  0.861708  0.452123  0.588963  0.834075  0.762968  0.644585   

        plays        to       too     watch  
문서1  0.000000  0.245112  0.322292  0.245112  
문서2  0.000000  0.343851  0.000000  0.343851  
문서3  0.680919  0.000000  0.000000  0.000000  
총합   0.680919  0.588963  0.322292  0.588963  


# Continuous Bag of Words (CBOW)


CBOW에 대해서 정리해보기
1.각 단어는 고차원(d) 벡터로 표현된다. (희소행렬보다는 적다.)
2.모든 단어는 행렬 V에 저장된다. 이 행렬의 크기는 N X D 이다. (N은 단어의 총수 D는 벡터의 차원)
3.CBOW모델에서 주변 단어들의 벡터를 합한 평균을 사용한다.  
$$
x = \sum_{i=1}^{n} v_i
$$

이걸 이용하는게  입력의. 기본원리
여기서 최대우도추정이나 이런 관점에서 접근할 수 있는데
최근에는 이걸 구하기 위해서 딥러닝을 이용하는 추세


저기 x에 w가중치를 곱하고 소프트맥스로 단어가 뭔지 예측하는게 기본형태


Skip-Gram 모델은 "sits"라는 단어를 입력으로 받아서 ("The", "cat", "on", "the", "mat")과 같은 문맥 단어들을 예측하는 데에 학습됩니다

 # 인코더 디코더 구현 나중에 해보기

완강(?)후기
정확하게는 강화학습 강의 5강으로 한시간정도 분량 남았는데,
그냥 미리 적어둔다.

입문용 강의인데 다루는 내용 폭을 보면...
누가봐도 어느정도 익숙한 사람을 위한 강의다.
CNN 다루는데 보통 이미지넷 발전과정만 다루지 보통 컴퓨터비전에서 다룰만한것도 상당히 많이 다루고

GAN이나 이런 부분에서도 그냥 간단하게 GAN이 무엇인지 이런식이랑만 다루지
온갖 종류의 GAN을 다루고..

솔직히 그냥 이거 커리큘럼 착실히 따라것 이해한다는 생각보다는 
"그냥 이런게 있구나 나중에 공부할 때 재미있겠는걸?" 같은 흐름잡이용으로 사용해야하는데,
너무 쓸데 없이 제대로 이해할거라고 계속 돌려보고 삽질함..
사실 GAN만 하더라도 여기서 오토인코더 50분 다룬 분량이 다른 특강으로는 3~4시간 분량 넘게 다룬다.

진짜 이미 알고 있거나 이런 상황이 아니면 제대로 이해하기 힘들다는 소리
그러니까 그냥 아는 사람이 복습용으로 사용하거나 "이런게 있구나" 하고 빠르게 듣고 
방향성 설정하는 강의

개인적으로 내가 여러가지 구현은 아니고 GPT 코드 보면서 이해하려고 노력한건 잘한 것 같다.
그래도 강의 듣는 습관이나 이런건 많이 좋아진 것 같네..
예전에는 진짜 아무런 생각 없이 들었는데..